In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
import asyncio
import nest_asyncio  # 이 두개는 jupyter lab에서만
nest_asyncio.apply() # 설정하는 함수입니다. py에선 사용안합니다.
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini")

async def main():
    async with MultiServerMCPClient(
        {
        "math": {
            "command": "python",
            "args": ["./server/math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            "command": "python",
            "args": ["./server/weather_server.py"],
            "transport": "stdio",
        },			
        "PDF_FAISS": {
            "command": "python",
            "args": ["./server/faiss_server.py"],
            "transport": "stdio",
        },

        }
    ) as client:
        agent = create_react_agent(model, client.get_tools())
        math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
        weather_response = await agent.ainvoke({"messages": "what is the weather in nyc?"})
        faiss_response = await agent.ainvoke({"messages": "langgraph에서 adaptive rag를 구축하는 방법을 알려주세요."})
        print(math_response)
        print(weather_response)
        print(faiss_response)
# asyncio 이벤트 루프를 통해 main() 실행
asyncio.run(main())

{'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='a0cf4475-eb91-4076-b570-84319b22d6e5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AQVSWvE4DsDYIAHYqydjyGTs', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 147, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b30da85f-9545-4230-9cdc-3743738f74a0-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_AQVSWvE4DsDYIAHYqydjyGTs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 147, 'output_tokens': 1